In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [3]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

In [5]:
X[0:3, :]

array([[619, 0, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 2, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 0, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57]], dtype=object)

In [6]:
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()

In [7]:
# Removing dummy variable, as [1, 0, 0] can be presented as [0,0]
X = X[:, 1:]

In [8]:
X[0:2, :]

array([[  0.00000000e+00,   0.00000000e+00,   6.19000000e+02,
          0.00000000e+00,   4.20000000e+01,   2.00000000e+00,
          0.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          1.00000000e+00,   1.01348880e+05],
       [  0.00000000e+00,   1.00000000e+00,   6.08000000e+02,
          0.00000000e+00,   4.10000000e+01,   1.00000000e+00,
          8.38078600e+04,   1.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   1.12542580e+05]])

In [9]:
# My way to get similar data above:
d_X = dataset.iloc[:, 3:13]
d_y = dataset.iloc[:, 13]

In [10]:
d_X = pd.get_dummies(d_X)

In [11]:
d_X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,1,0


In [12]:
d_X.drop(['Geography_France', 'Gender_Female'], axis=1, inplace=True)

In [13]:
d_X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [14]:
d_X.shape

(10000, 11)

In [15]:
X.shape

(10000, 11)

## Train Test Split

In [16]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [17]:
# Try my way: remove this if things go wrong later:
X_train, X_test, y_train, y_test = train_test_split(d_X, y, test_size = 0.2, random_state = 0)

## Feature scaling

In [18]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Part 2 - Now let's make the ANN!

In [64]:
import keras
from keras.models import Sequential
from keras.layers import Dense

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

# To make results reproduceable and setting gpu memory limit on tf.
# https://github.com/fchollet/keras/issues/2280

np.random.seed(123)
tf.set_random_seed(123)
config = tf.ConfigProto(inter_op_parallelism_threads=1)
config.gpu_options.per_process_gpu_memory_fraction = 0.1 # in my case this setting will use around 1G memory on GPU
set_session(tf.Session(config=config))



In [65]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
opt_adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) # 
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [66]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 100, epochs = 100, verbose=True)

Epoch 1/100
8000/8000 [==============================] - 0s - loss: 0.6727 - acc: 0.7935     
Epoch 2/100
8000/8000 [==============================] - 0s - loss: 0.5633 - acc: 0.7960     
Epoch 3/100
8000/8000 [==============================] - 0s - loss: 0.4571 - acc: 0.7960     
Epoch 4/100
8000/8000 [==============================] - 0s - loss: 0.4407 - acc: 0.7960     
Epoch 5/100
8000/8000 [==============================] - 0s - loss: 0.4361 - acc: 0.7960     
Epoch 6/100
8000/8000 [==============================] - 0s - loss: 0.4331 - acc: 0.7960     
Epoch 7/100
8000/8000 [==============================] - 0s - loss: 0.4311 - acc: 0.7960     
Epoch 8/100
8000/8000 [==============================] - 0s - loss: 0.4296 - acc: 0.7960     
Epoch 9/100
8000/8000 [==============================] - 0s - loss: 0.4287 - acc: 0.7960     
Epoch 10/100
8000/8000 [==============================] - 0s - loss: 0.4275 - acc: 0.7960     
Epoch 11/100
8000/8000 [==============================] - 0

8000/8000 [==============================] - 0s - loss: 0.4010 - acc: 0.8360     
Epoch 88/100
8000/8000 [==============================] - 0s - loss: 0.4010 - acc: 0.8351     
Epoch 89/100
8000/8000 [==============================] - 0s - loss: 0.4010 - acc: 0.8351     
Epoch 90/100
8000/8000 [==============================] - 0s - loss: 0.4008 - acc: 0.8356     
Epoch 91/100
8000/8000 [==============================] - 0s - loss: 0.4006 - acc: 0.8359     
Epoch 92/100
8000/8000 [==============================] - 0s - loss: 0.4006 - acc: 0.8359     
Epoch 93/100
8000/8000 [==============================] - 0s - loss: 0.4004 - acc: 0.8352     
Epoch 94/100
8000/8000 [==============================] - 0s - loss: 0.4005 - acc: 0.8361     
Epoch 95/100
8000/8000 [==============================] - 0s - loss: 0.4005 - acc: 0.8364     
Epoch 96/100
8000/8000 [==============================] - 0s - loss: 0.4005 - acc: 0.8350     
Epoch 97/100
8000/8000 [==============================] - 0s - 

# Part 3 - Making predictions and evaluating the model

In [67]:
# Predicting the Test set results
y_pred_pro = classifier.predict(X_test)
y_pred = (y_pred_pro > 0.5)

In [68]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [69]:
from sklearn.metrics import accuracy_score, roc_auc_score

accuracy_score(y_test, y_pred)

0.84299999999999997

In [70]:
roc_auc_score(y_test, y_pred_pro, average='macro')

0.79844421223731565

In [71]:
roc_auc_score(y_test, y_pred_pro, average='micro')

0.79844421223731565

## Use the model as if we have a new customer

In [33]:
new_customer = np.array([[600, 40, 3, 6000, 2, 1, 1, 50000,0, 0, 1]], dtype='float')
new_customer = sc.transform(new_customer)

In [34]:
new_customer

array([[-0.52111599,  0.10961719, -0.68538967, -1.11983631,  0.8095029 ,
         0.64259497,  0.9687384 , -0.87203322, -0.5698444 , -0.57369368,
         0.91601335]])

In [35]:
new_customer_pre = classifier.predict(new_customer)

In [69]:
new_customer_pre[0] > 0.5

array([False], dtype=bool)

## Using CV to evaluate

In [38]:
# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(0.3))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    # classifier.add(Dropout(0.3))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 100, epochs = 100, verbose=0)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = 1)
mean = accuracies.mean()
variance = accuracies.std()

In [39]:
mean

0.83437499627470968

In [40]:
variance

0.010250767245527004

## Parameter Turning

In [ ]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, verbose=0)


parameters = {'batch_size': [5, 25, 32, 50],
              'epochs': [50, 100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10, n_jobs=1, verbose=10 )
grid_search = grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)